In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


### Utilsier un modéle entrainée sur des données textuelles écrites en anglais ***all_minilm_l6-v2***

In [3]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Création et entrainement du modéle BERTopic

In [4]:
from bertopic import BERTopic

# Si vous voulez utiliser le modèle par défaut (anglais), vous pouvez omettre l'argument embedding_model
topic_model = BERTopic(
    embedding_model=embedding_model,  # Ou laissez vide pour défaut
    language="english",               # Indique la langue du texte
    verbose=True
)

topics, probs = topic_model.fit_transform(docs)


2025-01-22 13:38:52,629 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-01-22 13:39:58,399 - BERTopic - Embedding - Completed ✓
2025-01-22 13:39:58,405 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-22 13:40:32,004 - BERTopic - Dimensionality - Completed ✓
2025-01-22 13:40:32,006 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-22 13:40:32,071 - BERTopic - Cluster - Completed ✓
2025-01-22 13:40:32,101 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-22 13:40:32,314 - BERTopic - Representation - Completed ✓


### Liste des topics 

### Stop_words

## 1 

- Colonne docs

In [23]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP  # Seulement si vous souhaitez personnaliser UMAP

# ----- ÉTAPE A & B : Charger et nettoyer vos données -----

# 1) Lecture du fichier Excel
df = pd.read_excel("fichier_traduit.xlsx")

# Par exemple, vos colonnes textuelles sont 'presentation', 'activites', 'réponse1', 'réponse2'
colonnes_textuelles = ['presentation', 'activites', 'réponse1', 'réponse2']

# 2) Remplacer les NaN par des chaînes vides
df[colonnes_textuelles] = df[colonnes_textuelles].fillna("")

# 3) Supprimer les lignes entièrement vides (ou ne contenant que des espaces)
def est_vide_ou_espace(val):
    return val.strip() == ""

mask_ligne_vide = df[colonnes_textuelles].applymap(est_vide_ou_espace).all(axis=1)
df = df[~mask_ligne_vide].copy()

print("Nombre de lignes restantes après nettoyage :", len(df))

# 4) Créer la liste 'docs' en fusionnant les colonnes textuelles
docs = (
    df["presentation"] + " " +
    df["activites"]    + " " +
    df["réponse1"]     + " " +
    df["réponse2"]
).tolist()

# ----- ÉTAPE C : Entraîner BERTopic -----

# (Optionnel) Personnaliser UMAP si vous rencontrez des problèmes de dimension
umap_model = UMAP(
    n_neighbors=2,
    n_components=2,
    min_dist=0.0,
    metric='cosine'
)

# (Optionnel) Vectorizer pour supprimer les stopwords anglais
vectorizer_model = CountVectorizer(stop_words='english')

# Initialisation du modèle BERTopic
topic_model = BERTopic(
    language="english",                # Ajustez la langue si nécessaire
    vectorizer_model=vectorizer_model, # Retrait des stopwords
    umap_model=umap_model,             # Réduction de dimension personnalisée
    calculate_probabilities=False      # Mettez True si vous voulez les probs (et si votre dataset est assez grand)
)

# Entraînement du modèle
topics, probs = topic_model.fit_transform(docs)

# ----- AFFICHAGE DES TOPICS -----

# 1. Informations générales sur les topics
topic_info = topic_model.get_topic_info()
print("\n=== Aperçu des topics ===")
print(topic_info.head(10))

# 2. Mots-clés du premier topic
keywords_topic0 = topic_model.get_topic(0)
print("\n=== Mots-clés du topic 0 ===")
print(keywords_topic0)

# ----- VISUALISATIONS -----

# 1. Visualisation des topics (carte 2D via UMAP)
fig1 = topic_model.visualize_topics()
fig1.show()

# 2. Visualiser un diagramme en barres des topics
fig2 = topic_model.visualize_barchart(top_n_topics=5, n_words=5)
fig2.show()

# 3. Visualiser la hiérarchie des topics 
# 
# 

(dendrogramme)
fig3 = topic_model.visualize_hierarchy()
fig3.show()

# 4. Visualiser la matrice de similitude (heatmap) entre topics
fig4 = topic_model.visualize_heatmap()
fig4.show()


Nombre de lignes restantes après nettoyage : 272

=== Aperçu des topics ===
   Topic  Count                                     Name  \
0     -1     57             -1_artists_art_space_gallery   
1      0     44          0_art_contemporary_artists_arts   
2      1     24                 1_work_artists_art_space   
3      2     20           2_artists_franklin_work_pamela   
4      3     20             3_space_wall_artists_project   
5      4     19           4_matter_printed_artists_place   
6      5     16        5_art_activity_exhibition_artists   
7      6     13                    6_art_space_el_colona   
8      7     13       7_artists_kabinet_exhibition_brain   
9      8     12  8_association_exhibition_40mcube_sunset   

                                      Representation  \
0  [artists, art, space, gallery, new, media, con...   
1  [art, contemporary, artists, arts, artist, pro...   
2  [work, artists, art, space, international, exh...   
3  [artists, franklin, work, pamela, ar